In [4]:
import os
import pandas as pd
from sklearn import datasets, naive_bayes, svm, neighbors

# Задание 1
Исследуйте, как объем обучающей выборки и количество тестовых данных, влияет на точность классификации в датасетах про крестики-нолики (tic_tac_toe.txt) и о спаме e-mail сообщений (spam.csv) с помощью наивного Байесовского классификатора. Постройте графики зависимостей точности на обучающей и тестовой выборках в зависимости от их соотношения.

In [6]:
def load_tic_tac_toe():
    path = os.path.join(os.readlink('requisites'), 'tic_tac_toe.txt')
    df = pd.read_table(path)
    return df

In [8]:
def load_spam():
    path = os.path.join(os.readlink('requisites'), 'spam.csv')
    df = pd.read_table(path)
    return df

,",""make"",""address"",""all"",""num3d"",""our"",""over"",""remove"",""internet"",""order"",""mail"",""receive"",""will"",""people"",""report"",""addresses"",""free"",""business"",""email"",""you"",""credit"",""your"",""font"",""num000"",""money"",""hp"",""hpl"",""george"",""num650"",""lab"",""labs"",""telnet"",""num857"",""data"",""num415"",""num85"",""technology"",""num1999"",""parts"",""pm"",""direct"",""cs"",""meeting"",""original"",""project"",""re"",""edu"",""table"",""conference"",""charSemicolon"",""charRoundbracket"",""charSquarebracket"",""charExclamation"",""charDollar"",""charHash"",""capitalAve"",""capitalLong"",""capitalTotal"",""type"""
0,"1,0,0.64,0.64,0,0.32,0,0,0,0,0,0,0.64,0,0,0,0...."
1,"2,0.21,0.28,0.5,0,0.14,0.28,0.21,0.07,0,0.94,0..."
2,"3,0.06,0,0.71,0,1.23,0.19,0.19,0.12,0.64,0.25,..."
3,"4,0,0,0,0,0.63,0,0.31,0.63,0.31,0.63,0.31,0.31..."
4,"5,0,0,0,0,0.63,0,0.31,0.63,0.31,0.63,0.31,0.31..."
...,...
4596,"4597,0.31,0,0.62,0,0,0.31,0,0,0,0,0,1.88,0,0,0..."
4597,"4598,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,2..."
4598,"4599,0.3,0,0.3,0,0,0,0,0,0,0,0,1.8,0.3,0,0,0,0..."
4599,"4600,0.96,0,0,0,0.32,0,0,0,0,0,0,0.32,0,0,0,0,..."


# Задание 2
Сгенерируйте 100 точек с двумя признаками X1 и X2 в соответствии с нормальным распределением так, что одна и вторая часть точек (класс -1 и класс 1) имеют параметры: мат. ожидание X1, мат. ожидание X2, среднеквадратические отклонения для обеих переменных, соответствующие вашему варианту (указан в таблице). Построить диаграммы, иллюстрирующие данные. Построить Байесовский классификатор и оценить качество классификации с помощью различных методов (точность, матрица ошибок, ROСи PR-кривые). Является ли построенный классификатор «хорошим»?

# Задание 3
Постройте классификатор на основе метода k ближайших соседей для обучающего множества Glass (glass.csv). Посмотрите заголовки признаков и классов. Перед построением классификатора необходимо также удалить первый признак Id number, который не несет никакой информационной нагрузки.

## a.
Постройте графики зависимости ошибки классификации от количества ближайших соседей.

## b.
Определите подходящие метрики расстояния и исследуйте, как тип метрики расстояния влияет на точность классификации.

## c.
Определите, к какому типу стекла относится экземпляр с характеристиками: RI=1.516 Na=11.7 Mg=1.01 Al=1.19 Si=72.59 K=0.43 Ca=11.44 Ba=0.02 Fe=0.1

# Задание 4
Постройте классификаторы на основе метода опорных векторов для наборов данных из файлов svmdataN.txt и svmdataNtest.txt, где N – индекс задания:

## a.
Постройте алгоритм метода опорных векторов с линейным ядром. Визуализируйте разбиение пространства признаков на области с помощью полученной модели ([пример визуализации](https://scikit-learn.org/stable/auto_examples/svm/plot_iris_svc.html)). Выведите количество полученных опорных векторов, а также матрицу ошибок классификации на обучающей и тестовой выборках.

## b.
Постройте алгоритм метода опорных векторов с линейным ядром. Добейтесь нулевой ошибки сначала на обучающей выборке, а затем на тестовой, путем изменения штрафного параметра. Выберите оптимальное значение данного параметра и объясните свой выбор. Всегда ли нужно добиваться минимизации ошибки на обучающей выборке?

## c.
Постройте алгоритм метода опорных векторов, используя различные ядра (линейное, полиномиальное степеней 1-5, сигмоидальная функция, гауссово). Визуализируйте разбиение пространства признаков на области с помощью полученных моделей. Сделайте выводы.

## d.
Постройте алгоритм метода опорных векторов, используя различные ядра (полиномиальное степеней 1-5, сигмоидальная функция, гауссово). Визуализируйте разбиение пространства признаков на области с помощью полученных моделей. Сделайте выводы.

## e.
Постройте алгоритм метода опорных векторов, используя различные ядра (полиномиальное степеней 1-5, сигмоидальная функция, гауссово). Изменяя значение параметра ядра (гамма), продемонстрируйте эффект переобучения, выполните при этом визуализацию разбиения пространства признаков на области.

# Задание 5
Постройте классификаторы для различных данных на основе деревьев решений:

## a.
Загрузите набор данных Glass из файла glass.csv. Постройте дерево классификации для модели, предсказывающей тип (Type) по остальным признакам. Визуализируйте результирующее дерево решения. Дайте интерпретацию полученным результатам. Является ли построенное дерево избыточным? Исследуйте зависимость точности классификации от критерия расщепления, максимальной глубины дерева и других параметров по вашему усмотрению.

## b.
Загрузите набор данных spam7 из файла spam7.csv. Постройте оптимальное, по вашему мнению, дерево классификации для параметра yesno. Объясните, как был осуществлён подбор параметров. Визуализируйте результирующее дерево решения.Определите наиболее влияющие признаки. Оцените качество классификации.

# Задание 6
Загрузите набор данных из файла bank_scoring_train.csv. Это набор финансовых данных, характеризующий физических лиц. Целевым столбцом является «SeriousDlqin2yrs», означающий, ухудшится ли финансовая ситуация у клиента. Постройте систему по принятию решения о выдаче или невыдаче кредита физическому лицу. Сделайте как минимум 2 варианта системы на основе различных классификаторов. Подберите подходящую метрику качества работы системы исходя из специфики задачи и определите, принятие решения какой системой сработало лучше на bank_scoring_test.csv.